In [0]:
pip install pyspark

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached pyspark-3.5.0-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
spark.conf.set(
    "fs.azure.account.key.datacohortworkspacelabs.dfs.core.windows.net",
    "NUwhjFcHG95EU1Rnu7+Woq3JQP28bXy5kDQhA9yFV68XBz1umr7uqeQgMhrwxHfTwNWxAx/n1K6j+AStGTUMkQ==")

In [0]:
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import lit, current_timestamp, input_file_name, expr

landing_path = "abfss://landing-kay@datacohortworkspacelabs.dfs.core.windows.net/yellow_tripdata_2019-*"
raw_data = spark.read.format("csv").option("header", "true").load(landing_path)

raw_data = raw_data.withColumn("tripID", expr("uuid()")) \
                             .withColumn("FileName", input_file_name()) \
                             .withColumn("CreatedOn", current_timestamp())

bronze_path = "abfss://bronze-kay@datacohortworkspacelabs.dfs.core.windows.net/"
raw_data.write.partitionBy("VendorID").mode("overwrite").format("delta").option("overwriteSchema", "true").save(bronze_path)

spark.stop()